In [1]:
import requests
from lxml import etree
import pandas as pd
from key import getServiceKey

__SERVICE_KEY__ = getServiceKey()   # 서비스키 받기

def iter_nodes(element):
    # 어떤 공공데이터도 이 함수로 파싱 가능
    # Recursive - Depth First Search하여 XML 데이터 구조 저장 (JSON 형태로 저장)
    result = {}
    if len(element)>0:  # Empty Node가 아닌 경우
        children_list = sorted(list(set([x.tag for x in element.getchildren()])))   # 자식 노드 (Unique values)
        for children in children_list:
            siblings = element.findall(children)
            if len(siblings)>1:
                sibling_list = []
                for i in siblings:
                    sibling_list.append(iter_nodes(i))
                result[str(children)] = sibling_list
            else:
                result[str(children)] = iter_nodes(siblings[0])
        return result
    else:   # Empty Node인 경우
        return element.text


In [2]:
region_dictionary = {
   "서울특별시":"11000",
   "서울특별시 종로구":"11110",
   "서울특별시 중구":"11140",
   "서울특별시 용산구":"11170",
   "서울특별시 성동구":"11200",
   "서울특별시 광진구":"11215",
   "서울특별시 동대문구":"11230",
   "서울특별시 중랑구":"11260",
   "서울특별시 성북구":"11290",
   "서울특별시 강북구":"11305",
   "서울특별시 도봉구":"11320",
   "서울특별시 노원구":"11350",
   "서울특별시 은평구":"11380",
   "서울특별시 서대문구":"11410",
   "서울특별시 마포구":"11440",
   "서울특별시 양천구":"11470",
   "서울특별시 강서구":"11500",
   "서울특별시 구로구":"11530",
   "서울특별시 금천구":"11545",
   "서울특별시 영등포구":"11560",
   "서울특별시 동작구":"11590",
   "서울특별시 관악구":"11620",
   "서울특별시 서초구":"11650",
   "서울특별시 강남구":"11680",
   "서울특별시 송파구":"11710",
   "서울특별시 강동구":"11740",
   "부산광역시":"26000",
   "부산광역시 중구":"26110",
   "부산광역시 서구":"26140",
   "부산광역시 동구":"26170",
   "부산광역시 영도구":"26200",
   "부산광역시 부산진구":"26230",
   "부산광역시 동래구":"26260",
   "부산광역시 남구":"26290",
   "부산광역시 북구":"26320",
   "부산광역시 해운대구":"26350",
   "부산광역시 사하구":"26380",
   "부산광역시 금정구":"26410",
   "부산광역시 강서구":"26440",
   "부산광역시 연제구":"26470",
   "부산광역시 수영구":"26500",
   "부산광역시 사상구":"26530",
   "부산광역시 기장군":"26710",
   "대구광역시":"27000",
   "대구광역시 중구":"27110",
   "대구광역시 동구":"27140",
   "대구광역시 서구":"27170",
   "대구광역시 남구":"27200",
   "대구광역시 북구":"27230",
   "대구광역시 수성구":"27260",
   "대구광역시 달서구":"27290",
   "대구광역시 달성군":"27710",
   "인천광역시":"28000",
   "인천광역시 중구":"28110",
   "인천광역시 동구":"28140",
   "인천광역시 미추홀구":"28177",
   "인천광역시 연수구":"28185",
   "인천광역시 남동구":"28200",
   "인천광역시 부평구":"28237",
   "인천광역시 계양구":"28245",
   "인천광역시 서구":"28260",
   "인천광역시 강화군":"28710",
   "인천광역시 옹진군":"28720",
   "광주광역시":"29000",
   "광주광역시 동구":"29110",
   "광주광역시 서구":"29140",
   "광주광역시 남구":"29155",
   "광주광역시 북구":"29170",
   "광주광역시 광산구":"29200",
   "대전광역시":"30000",
   "대전광역시 동구":"30110",
   "대전광역시 중구":"30140",
   "대전광역시 서구":"30170",
   "대전광역시 유성구":"30200",
   "대전광역시 대덕구":"30230",
   "울산광역시":"31000",
   "울산광역시 중구":"31110",
   "울산광역시 남구":"31140",
   "울산광역시 동구":"31170",
   "울산광역시 북구":"31200",
   "울산광역시 울주군":"31710",
   "세종특별자치시":"36110",
   "경기도":"41000",
   "경기도 수원시":"41110",
   "경기도 수원시 장안구":"41111",
   "경기도 수원시 권선구":"41113",
   "경기도 수원시 팔달구":"41115",
   "경기도 수원시 영통구":"41117",
   "경기도 성남시":"41130",
   "경기도 성남시 수정구":"41131",
   "경기도 성남시 중원구":"41133",
   "경기도 성남시 분당구":"41135",
   "경기도 의정부시":"41150",
   "경기도 안양시":"41170",
   "경기도 안양시 만안구":"41171",
   "경기도 안양시 동안구":"41173",
   "경기도 부천시":"41190",
   "경기도 광명시":"41210",
   "경기도 평택시":"41220",
   "경기도 동두천시":"41250",
   "경기도 안산시":"41270",
   "경기도 안산시 상록구":"41271",
   "경기도 안산시 단원구":"41273",
   "경기도 고양시":"41280",
   "경기도 고양시 덕양구":"41281",
   "경기도 고양시 일산동구":"41285",
   "경기도 고양시 일산서구":"41287",
   "경기도 과천시":"41290",
   "경기도 구리시":"41310",
   "경기도 남양주시":"41360",
   "경기도 오산시":"41370",
   "경기도 시흥시":"41390",
   "경기도 군포시":"41410",
   "경기도 의왕시":"41430",
   "경기도 하남시":"41450",
   "경기도 용인시":"41460",
   "경기도 용인시 처인구":"41461",
   "경기도 용인시 기흥구":"41463",
   "경기도 용인시 수지구":"41465",
   "경기도 파주시":"41480",
   "경기도 이천시":"41500",
   "경기도 안성시":"41550",
   "경기도 김포시":"41570",
   "경기도 화성시":"41590",
   "경기도 광주시":"41610",
   "경기도 양주시":"41630",
   "경기도 포천시":"41650",
   "경기도 여주시":"41670",
   "경기도 연천군":"41800",
   "경기도 가평군":"41820",
   "경기도 양평군":"41830",
   "강원도":"42000",
   "강원도 춘천시":"42110",
   "강원도 원주시":"42130",
   "강원도 강릉시":"42150",
   "강원도 동해시":"42170",
   "강원도 태백시":"42190",
   "강원도 속초시":"42210",
   "강원도 삼척시":"42230",
   "강원도 홍천군":"42720",
   "강원도 횡성군":"42730",
   "강원도 영월군":"42750",
   "강원도 평창군":"42760",
   "강원도 정선군":"42770",
   "강원도 철원군":"42780",
   "강원도 화천군":"42790",
   "강원도 양구군":"42800",
   "강원도 인제군":"42810",
   "강원도 고성군":"42820",
   "강원도 양양군":"42830",
   "충청북도":"43000",
   "충청북도 청주시":"43110",
   "충청북도 청주시 상당구":"43111",
   "충청북도 청주시 서원구":"43112",
   "충청북도 청주시 흥덕구":"43113",
   "충청북도 청주시 청원구":"43114",
   "충청북도 충주시":"43130",
   "충청북도 제천시":"43150",
   "충청북도 보은군":"43720",
   "충청북도 옥천군":"43730",
   "충청북도 영동군":"43740",
   "충청북도 증평군":"43745",
   "충청북도 진천군":"43750",
   "충청북도 괴산군":"43760",
   "충청북도 음성군":"43770",
   "충청북도 단양군":"43800",
   "충청남도":"44000",
   "충청남도 천안시":"44130",
   "충청남도 천안시 동남구":"44131",
   "충청남도 천안시 서북구":"44133",
   "충청남도 공주시":"44150",
   "충청남도 보령시":"44180",
   "충청남도 아산시":"44200",
   "충청남도 서산시":"44210",
   "충청남도 논산시":"44230",
   "충청남도 계룡시":"44250",
   "충청남도 당진시":"44270",
   "충청남도 금산군":"44710",
   "충청남도 부여군":"44760",
   "충청남도 서천군":"44770",
   "충청남도 청양군":"44790",
   "충청남도 홍성군":"44800",
   "충청남도 예산군":"44810",
   "충청남도 태안군":"44825",
   "전라북도":"45000",
   "전라북도 전주시":"45110",
   "전라북도 전주시 완산구":"45111",
   "전라북도 전주시 덕진구":"45113",
   "전라북도 군산시":"45130",
   "전라북도 익산시":"45140",
   "전라북도 정읍시":"45180",
   "전라북도 남원시":"45190",
   "전라북도 김제시":"45210",
   "전라북도 완주군":"45710",
   "전라북도 진안군":"45720",
   "전라북도 무주군":"45730",
   "전라북도 장수군":"45740",
   "전라북도 임실군":"45750",
   "전라북도 순창군":"45770",
   "전라북도 고창군":"45790",
   "전라북도 부안군":"45800",
   "전라남도":"46000",
   "전라남도 목포시":"46110",
   "전라남도 여수시":"46130",
   "전라남도 순천시":"46150",
   "전라남도 나주시":"46170",
   "전라남도 광양시":"46230",
   "전라남도 담양군":"46710",
   "전라남도 곡성군":"46720",
   "전라남도 구례군":"46730",
   "전라남도 고흥군":"46770",
   "전라남도 보성군":"46780",
   "전라남도 화순군":"46790",
   "전라남도 장흥군":"46800",
   "전라남도 강진군":"46810",
   "전라남도 해남군":"46820",
   "전라남도 영암군":"46830",
   "전라남도 무안군":"46840",
   "전라남도 함평군":"46860",
   "전라남도 영광군":"46870",
   "전라남도 장성군":"46880",
   "전라남도 완도군":"46890",
   "전라남도 진도군":"46900",
   "전라남도 신안군":"46910",
   "경상북도":"47000",
   "경상북도 포항시":"47110",
   "경상북도 포항시 남구":"47111",
   "경상북도 포항시 북구":"47113",
   "경상북도 경주시":"47130",
   "경상북도 김천시":"47150",
   "경상북도 안동시":"47170",
   "경상북도 구미시":"47190",
   "경상북도 영주시":"47210",
   "경상북도 영천시":"47230",
   "경상북도 상주시":"47250",
   "경상북도 문경시":"47280",
   "경상북도 경산시":"47290",
   "경상북도 군위군":"47720",
   "경상북도 의성군":"47730",
   "경상북도 청송군":"47750",
   "경상북도 영양군":"47760",
   "경상북도 영덕군":"47770",
   "경상북도 청도군":"47820",
   "경상북도 고령군":"47830",
   "경상북도 성주군":"47840",
   "경상북도 칠곡군":"47850",
   "경상북도 예천군":"47900",
   "경상북도 봉화군":"47920",
   "경상북도 울진군":"47930",
   "경상북도 울릉군":"47940",
   "경상남도":"48000",
   "경상남도 창원시":"48120",
   "경상남도 창원시 의창구":"48121",
   "경상남도 창원시 성산구":"48123",
   "경상남도 창원시 마산합포구":"48125",
   "경상남도 창원시 마산회원구":"48127",
   "경상남도 창원시 진해구":"48129",
   "경상남도 진주시":"48170",
   "경상남도 통영시":"48220",
   "경상남도 사천시":"48240",
   "경상남도 김해시":"48250",
   "경상남도 밀양시":"48270",
   "경상남도 거제시":"48310",
   "경상남도 양산시":"48330",
   "경상남도 의령군":"48720",
   "경상남도 함안군":"48730",
   "경상남도 창녕군":"48740",
   "경상남도 고성군":"48820",
   "경상남도 남해군":"48840",
   "경상남도 하동군":"48850",
   "경상남도 산청군":"48860",
   "경상남도 함양군":"48870",
   "경상남도 거창군":"48880",
   "경상남도 합천군":"48890",
   "제주특별자치도":"50000",
   "제주특별자치도 제주시":"50110",
   "제주특별자치도 서귀포시":"50130"
}

In [5]:
region_name = [
    "서울특별시 종로구",
    "서울특별시 중구",
    "서울특별시 용산구",
    "서울특별시 성동구",
    "서울특별시 광진구",
    "서울특별시 동대문구",
    "서울특별시 중랑구",
    "서울특별시 성북구",
    "서울특별시 강북구",
    "서울특별시 도봉구",
    "서울특별시 노원구",
    "서울특별시 은평구",
    "서울특별시 서대문구",
    "서울특별시 마포구",
    "서울특별시 양천구",
    "서울특별시 강서구",
    "서울특별시 구로구",
    "서울특별시 금천구",
    "서울특별시 영등포구",
    "서울특별시 동작구",
    "서울특별시 관악구",
    "서울특별시 서초구",
    "서울특별시 강남구",
    "서울특별시 송파구",
    "서울특별시 강동구",
    "경기도 성남시 수정구",
    "경기도 성남시 중원구",
    "경기도 성남시 분당구",
    "경기도 과천시",
    "경기도 구리시",
    "경기도 남양주시",
    "경기도 하남시"
]
time = ["202111", "202112", "202201", "202202"]

df="str"
for t in time:
    for r in region_name:
        url = 'http://openapi.molit.go.kr:8081/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcRHTrade'
        params ={'serviceKey' : __SERVICE_KEY__, 'LAWD_CD' : region_dictionary[r], 'DEAL_YMD' : t}
        response = requests.get(url, params=params)
        currentNode = etree.fromstring(response.content) # 현재 root 가리킴
        result = iter_nodes(currentNode)
        

        for item in result['body']['items']['item']:
            item["시군구"] = r

        if type(df)==str:
            df = pd.DataFrame(result['body']['items']['item'])
        else:
            temp = pd.DataFrame(result['body']['items']['item'])
            df = pd.concat([df, temp])

In [6]:
df = df[df["해제여부"]!='O']

df = df.astype({'건축년도':'int'})
df = df[df["건축년도"]<1997]
df

,거래금액,거래유형,건축년도,년,대지권면적,법정동,연립다세대,월,일,전용면적,중개사소재지,지번,지역코드,층,해제사유발생일,해제여부,시군구
5,"26,500",중개거래,1988,2021,58.73,사직동,(262-141),11,18,35.88,서울 종로구,262-141,11110,1,,,서울특별시 종로구
8,"50,000",중개거래,1993,2021,42.49,가회동,반석빌라,11,25,77.76,서울 종로구,33-19,11110,2,,,서울특별시 종로구
9,"30,000",중개거래,1993,2021,20.25,연건동,상일빌라,11,23,38.27,서울 종로구,301-2,11110,-1,,,서울특별시 종로구
27,"41,000",중개거래,1993,2021,17,행촌동,(210-121),11,3,33.84,서울 마포구,210-121,11110,3,,,서울특별시 종로구
29,"20,000",중개거래,1996,2021,14.6,행촌동,(210-219),11,11,40.19,서울 종로구,210-219,11110,1,,,서울특별시 종로구
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161,"26,000",중개거래,1987,2022,52.24,퇴계원읍 퇴계원리,"우남연립(라,마동)",2,11,49.56,경기 남양주시,299-5,41360,3,,,경기도 남양주시
163,"15,000",중개거래,1992,2022,30.6,퇴계원읍 퇴계원리,삼영빌라,2,21,39.78,서울 광진구,218-118,41360,1,,,경기도 남양주시
164,"12,500",중개거래,1993,2022,25.923,퇴계원읍 퇴계원리,"대림하이츠5,6동",2,22,36.48,경기 남양주시,97-32,41360,3,,,경기도 남양주시
165,"14,600",직거래,1990,2022,30.37,퇴계원읍 퇴계원리,(180-7),2,25,38.21,,180-7,41360,1,,,경기도 남양주시


In [7]:
import numpy as np
import pandas as pd
from urllib.request import urlopen
from urllib import parse
from urllib.request import Request
from urllib.error import HTTPError
from bs4 import BeautifulSoup
import json

#naver map api key
client_id = 'qwp1vycbgo';    # 본인이 할당받은 ID 입력
client_pw = '3rVsuYkeUHdORNF4rhqnsgawYpfUkyQufatZ1dXg';    # 본인이 할당받은 Secret 입력

api_url = 'https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode?query='

data = df['시군구'] + ' ' + df['법정동'] + ' ' + df['지번']

# 네이버 지도 API 이용해서 위경도 찾기
geo_coordi = []     
for add in data:
    add_urlenc = parse.quote(add)  
    url = api_url + add_urlenc
    request = Request(url)
    request.add_header('X-NCP-APIGW-API-KEY-ID', client_id)
    request.add_header('X-NCP-APIGW-API-KEY', client_pw)
    try:
        response = urlopen(request)
    except HTTPError as e:
        print('HTTP Error!')
        latitude = None
        longitude = None
    else:
        rescode = response.getcode()
        if rescode == 200:
            response_body = response.read().decode('utf-8')
            response_body = json.loads(response_body)   # json
            if response_body['addresses'] == [] :
                print("'result' not exist!")
                latitude = None
                longitude = None
            else:
                latitude = response_body['addresses'][0]['y']
                longitude = response_body['addresses'][0]['x']
        else:
            print('Response error code : %d' % rescode)
            latitude = None
            longitude = None

    geo_coordi.append([latitude, longitude])


np_geo_coordi = np.array(geo_coordi)
pd_geo_coordi = pd.DataFrame({"도로명": data.values,
                              "위도": np_geo_coordi[:, 0],
                              "경도": np_geo_coordi[:, 1],
                              "거래": 1,
                              })

In [8]:
pd_geo_coordi


,도로명,위도,경도,거래
0,서울특별시 종로구 사직동 262-141,37.5747502,126.9657123,1
1,서울특별시 종로구 가회동 33-19,37.5822396,126.9833461,1
2,서울특별시 종로구 연건동 301-2,37.5765720,126.9991517,1
3,서울특별시 종로구 행촌동 210-121,37.5746695,126.9620793,1
4,서울특별시 종로구 행촌동 210-219,37.5743067,126.9616314,1
...,...,...,...,...
3126,경기도 남양주시 퇴계원읍 퇴계원리 299-5,37.6497756,127.1400865,1
3127,경기도 남양주시 퇴계원읍 퇴계원리 218-118,37.6482485,127.1418823,1
3128,경기도 남양주시 퇴계원읍 퇴계원리 97-32,37.6551549,127.1428169,1
3129,경기도 남양주시 퇴계원읍 퇴계원리 180-7,37.6470731,127.1437013,1


In [9]:
import folium

f = folium.Figure(width=1600, height=1200)

map_osm = folium.Map(
    location=[
        (pd_geo_coordi["위도"].astype(float).max() + pd_geo_coordi["위도"].astype(float).min())/2.0, 
        (pd_geo_coordi["경도"].astype(float).max()+pd_geo_coordi["경도"].astype(float).min())/2.0
        ],
    zoom_start=14,
    )

f.add_child(map_osm)

#for index, row in pd_geo_coordi.iterrows():
#    folium.CircleMarker(location=[row["위도"], row["경도"]], radius=2, ).add_to(map_osm)
#f

In [10]:
from folium.plugins import HeatMap

_list = []
for i in pd_geo_coordi.iterrows():
    _list.append([float(i[1]["위도"]), float(i[1]["경도"])])
_list


[[37.5747502, 126.9657123],
 [37.5822396, 126.9833461],
 [37.576572, 126.9991517],
 [37.5746695, 126.9620793],
 [37.5743067, 126.9616314],
 [37.5747757, 126.9631047],
 [37.6058334, 126.955286],
 [37.6051936, 126.9564612],
 [37.6097958, 126.9560453],
 [37.6077924, 126.9709948],
 [37.6059226, 126.9654269],
 [37.6089175, 126.976605],
 [37.5561413, 126.9798764],
 [37.560367, 127.0080829],
 [37.5521938, 127.0062062],
 [37.5569671, 127.0163215],
 [37.5519135, 127.0056404],
 [37.5434305, 126.9872769],
 [37.5492127, 126.9647632],
 [37.5502061, 126.9663127],
 [37.5485981, 126.9657207],
 [37.5477819, 126.9653282],
 [37.5439287, 126.9672715],
 [37.5413071, 126.9663668],
 [37.540117, 126.9657871],
 [37.5418163, 126.9604493],
 [37.5420727, 126.9642068],
 [37.5264344, 126.9548301],
 [37.5266805, 126.9542827],
 [37.5362028, 126.9893071],
 [37.5382201, 126.9928468],
 [37.5390659, 126.992697],
 [37.5372735, 126.9906858],
 [37.533084, 126.9990954],
 [37.5302049, 127.0028459],
 [37.5613361, 127.0470174],

In [12]:
HeatMap(_list).add_to(map_osm)
f

In [123]:
f.save('재개발투기지도.html')